In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import regex as re
import time
import pickle

In [2]:
recordF=pd.read_csv("../input/categories/recordF.csv")

In [3]:
recordF.head()

,document_id,status,document_url,media_type,media_link,relative_media_path,is_media_downloaded,transcription_text,transcription_text_relative_path,from_time,to_time,project_id,Category,Unnamed: 13,Unnamed: 14
0,NMAAHC-004567391_00463,Complete,https://transcription.si.edu/transcribe/9650/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9650/image/NMAAHC-004567391_00463.jpg,1,[C 108 ENCL]\r\n\r\nQuarter ending March 31st ...,/9650/transcriptions/NMAAHC-004567391_00463.txt,\N,\N,9650,Records - Financial,NaN,NaN
1,NMAAHC-004567391_00464,Complete,https://transcription.si.edu/transcribe/9650/N...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/9650/image/NMAAHC-004567391_00464.jpg,1,[C108ENCL]\r\n\r\nReport of Monies rec'd in Di...,/9650/transcriptions/NMAAHC-004567391_00464.txt,\N,\N,9650,Records - Financial,NaN,NaN
2,NMAAHC-004567415_00348,Complete,https://transcription.si.edu/transcribe/15411/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15411/image/NMAAHC-004567415_00348.jpg,1,N. Carolina\r\n\r\nMemorandum of Expenses at L...,/15411/transcriptions/NMAAHC-004567415_00348.txt,\N,\N,15411,Records - Financial,NaN,NaN
3,NMAAHC-004567415_00376,Complete,https://transcription.si.edu/transcribe/15411/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15411/image/NMAAHC-004567415_00376.jpg,1,"Mem ""A""\r\n\r\nPurchases in November as per Co...",/15411/transcriptions/NMAAHC-004567415_00376.txt,\N,\N,15411,Records - Financial,NaN,NaN
4,NMAAHC-004567415_00377,Complete,https://transcription.si.edu/transcribe/15411/...,image,https://ids.si.edu/ids/deliveryService?max_w=&...,/15411/image/NMAAHC-004567415_00377.jpg,1,Apparent indebtedness not paid $94.19\r\nDedu...,/15411/transcriptions/NMAAHC-004567415_00377.txt,\N,\N,15411,Records - Financial,NaN,NaN


In [4]:
recordF.shape

(3599, 15)

In [5]:
recordF_text=recordF["transcription_text"]

In [6]:
print(recordF_text[0])

[C 108 ENCL]

Quarter ending March 31st 1866
Justin Hodge Capt & A.Q.M. and Financial Agent
Bureau Refugees Freedmen and Abandoned Lands
Dist West. N.Ca.

[[double line across page]]
[[table with lines drawn for columns]]

Date | Of Whom Received | For what | Amount Dolls | Amount Cts

1865
Dec 18" | Henry Swim | One house at Greensboro N.C. | 78 | 00
[[ditto for: Dec]] 28 | Wm B Meins | one open shed at Nottsburg Sta | 25 | 00
1866
Jany 3d | J.W. Hall | Brick on Prison ground at Salisbury | 200 | 00
[[ditto for: Jany]] 8" | Capt JC Barnett | Fines at Charlotte N.C. | 10 | 00
[[ditto for: Jany]] 23 | Capt FA Seeley AQM | Draft on US Treasury New York | 822 | 00
Feby 2d | Capt JC Barnett | Fines at Charlotte N.C. | 20 | 00
[[ditto for: Feby]] 13th | Wm Valentine | One frame at Salisbury N.C. | 30 | 00
[[ditto for: Feby]] 17" | DA Davis | One frame at China Grove Sta | 23 | 00
[[ditto for: Feby]] 22d | Jacob Bodecker | Rents at Greensboro N.C. | 23 | 00
Mar 8" | Capt J.C. Barnett | Fines

In [7]:
print(recordF_text[10])

260
Fodder divided - 15 Stacks ea to J 75.00
Shucks divided 22.70  22.70
[[tally line]]
97.70

27 1/2 250
18 75.
[[tally line]] 
45

455 56                67.81               52
48.00                 33.90
[[tally line]]       [[tally line]]       385.48
407.56               101.71               237.33
237.97                                    [[tally line]]
[[tally line]]                           $148.15
169.59
[[tally line]]
67.81
33 90
[[tally line]] $101.71      [[?]]10
92.51                        114
[[tally line]]                 5
9.20                        [[tally line]]
                              570     
                              150
                           [[tally line]]
                             571 50

7) 474.69
[[tally line]]
67.81
3 1/2
[[tally line]]
203 43
33 90
[[tally line]]
237.33

Joe 
Levi
Thad
Jack 1/2

Jac full hand
John full hand 
Matthew full hand
Phil full hand
Levi [[checkmark]] full hand
Thad [[checkmark]] full hand
Audrey. & Jack full hand


In [8]:
print(recordF_text[100])

Triplicate Bill of Lading.

Lincolnton N.C. October, 1867. Received from Bvt. Lieut Col. Thos. P. Johnston A. Q. M. United States Army, the following articles of public property, as specified below, (contents and value unknown,) in apparent good order, to be forwarded to  by the Wilmington Charlotte & Rutherford Rail Roads and connecting line there to be delivered in like good order and condition, unto Lieut. T. D. McAlpine Sub Asst Commissioner. Freight to be paid by Disbursing Quartermaster U.S.A. at Raleigh N.C., and on the original Bill of Lading at Authorized rates.
Jno G Justice
Agt

Payable by Freedman's BUREAU

[[Five columned table]]
Marks. | Nos. | Packages, &c., | Contents. | Pd. Weight

Lt. T. D. McAlpine Charlotte N C | 1 | Desk |   | { 165

Lt. T. D. McAlpine Charlotte N C | 1 | Table |   | { 165

Lt. T. D. McAlpine Charlotte N C | 4 | Chairs |   | { 165
[[/Five columned table]]



In [9]:
print(recordF_text[500])

Dec. 14/67
George Chavis
Receipt
for
$9.18


In [10]:
print(recordF_text[700])

Oxford N.C. June 22 & 1868
Received of E.J. Lamberton Agent Bu R.F. and AL one 50/100 dollars being amount of judgement against Amos Harris F in favor of Ann McLaddin 
his
Addison x McLaddin
mark 

Witness
R.S. Lunt


In [11]:
print(recordF_text[1005])

Galveston, Texas

Received of [[crossed out]] Rev. JOS. WELCH, [[/crossed out]] Superintendent of Education, Bureau of Refugees, Freedmen and Abandoned Lands, State of Texas, General Balloch's Check No 8632, for $10 00/100, being in payment of services as teacher for the month of February, [[crossed out]] 1869 [[/crossed out]] 1870.

John O. Stevenson


In [12]:
print(recordF_text[1300])

4th Case James Sides Col. }
vs                        } for debt
George M Sides white      }

George M. Sides white     }
vs                        } for violation of contract
James Sides Col.          }
parties met before me & Setted


5th Case H. T. Biles admr. of Nelson Pennington Jr 
vs               }    
James Sides col. } for one horse Eleven dollars & 50 cts
Sides colerd to pay Biles for horse


6th Thomas Biles white }
vs                     } for abuse and slanderous 
Hager colord           } talk
Setled by Hagers promising to do so no more


7th Daniel Shankles wife col } 
vs                           } for abuse
Daniel Shankle her husband   }
I dont know what to do with the case

There has bin several Bastard cases. I dont no what to do withe them please Give me some instructions. [[strikethrough]] U.S. Graves all [[/strikethrough]] U.S. Soldiers Graves all Respected 
No Rations issued to Freedmen
Freedmen working tolerable well
the above is all Respectfully Submitted for t

In [13]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()
stop_words.remove("of")
punct='!"#$%&\'()*+-/<=>?@[\\]^_`{|}~'
def process(s):
    s= re.sub("([\[]).*?([\]])"," ",s)
    for p in punct:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    return s
for i in tqdm(range(len(recordF_text))):
    recordF_text[i] = process(recordF_text[i])
for i in tqdm(range(len(recordF_text))):
    recordF_text[i] = " ".join(recordF_text[i])

100%|██████████| 3599/3599 [00:01<00:00, 2196.27it/s]


In [14]:
recordF_text

0       Quarter ending March 31st 1866 Justin Hodge Ca...
1       Report of Monies recd Dist W NC previously rep...
2       N. Carolina Memorandum of Expenses Linwood Dav...
3       Mem A Purchases November per Cooks books 55.20...
4       Apparent indebtedness paid 94.19 Deduct Balanc...
                              ...                        
3594                                                    N
3595                                                    N
3596                                                    N
3597    Recpto Wm M. Waters 1895 proportionate Govt po...
3598    Bureau R. F. A. L. Wilmington N C Nov 1665 Rec...
Name: transcription_text, Length: 3599, dtype: object

In [15]:
recordF_text[700]

'Oxford N.C. June 22 1868 Received of E.J . Lamberton Agent Bu R.F . AL one 50100 dollars amount of judgement Amos Harris F favor of Ann McLaddin Addison x McLaddin mark Witness R.S . Lunt'

In [16]:
!pip3 install Cython
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.

In [17]:
from flair.models import SequenceTagger

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-28 07:47:32,411 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-28 07:48:01,875 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [18]:
from flair.models import SequenceTagger

tagger1 = SequenceTagger.load("flair/ner-english")

Downloading:   0%|          | 0.00/432M [00:00<?, ?B/s]

2022-06-28 07:48:18,537 loading file /root/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-06-28 07:48:20,362 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [19]:
from flair.models import SequenceTagger

tagger2 = SequenceTagger.load("flair/ner-english-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-28 07:49:39,939 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
2022-06-28 07:49:59,317 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


# ontonotes

In [20]:
lst=[]
label=[]
Per=[]
Date=[]
GPE=[]
ORG=[]
LOC=[]
Bulding=[]
Cardinal=[]
from flair.data import Sentence

recordF_onto=pd.DataFrame({"recordF_text":"", "Name-ontonotes":[], "GPE-ontonotes":"","ORG-ontonotes":"", "Date-ontonotes":""})
for i in range(3599):
        sentencess =Sentence(recordF_text[i])
        recordF_onto.loc[i,"recordF_text"]=recordF_text[i]
        tagger.predict(sentencess)
        dc={}
        P=[]
        D=[]
        G=[]
        L=[]
        O=[]
        B=[]
        C=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.97) & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PERSON":
                      P.append(j)
                elif dc[j]=="ORG":
                      O.append(j)
                elif dc[j]=="DATE":
                      D.append(j)
                elif dc[j]=="GPE":
                      G.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
                elif dc[j]=="FAC":
                      B.append(j)
                elif dc[j]=="CARDINAL":
                      C.append(j)
        Per.append(P)
        Date.append(D)
        ORG.append(O)
        GPE.append(G)
        LOC.append(L)
        Bulding.append(B)
        Cardinal.append(C)
        
            
recordF_onto["Date-ontonotes"]=Date
recordF_onto["Name-ontonotes"]=Per
recordF_onto["GPE-ontonotes"]=GPE
recordF_onto["ORG-ontonotes"]=ORG 
recordF_onto["LOC-ontonotes"]=LOC 
recordF_onto["Building-ontonotes"]=Bulding
recordF_onto["Cardinal-ontonotes"]=Cardinal

In [21]:
recordF_onto["recordF_text"]=recordF_text

In [22]:
recordF_onto.head()

,recordF_text,Name-ontonotes,GPE-ontonotes,ORG-ontonotes,Date-ontonotes,LOC-ontonotes,Building-ontonotes,Cardinal-ontonotes
0,Quarter ending March 31st 1866 Justin Hodge Ca...,"[Justin Hodge, Henry Swim, JC Barnett, Seeley,...","[Nottsburg, Salisbury]",[US Treasury],"[Quarter ending March 31st 1866, Dec 18, Jany ...",[],[],[One]
1,Report of Monies recd Dist W NC previously rep...,"[Cilley, Dag Palmer 2.00, Clinton A Cilley]",[],[AAG],[],[],[],"[25, 15.00, 10.00, 27.00]"
2,N. Carolina Memorandum of Expenses Linwood Dav...,"[G . W. Myers, R . Lanier, C . Davis 1, W.R . ...","[N. Carolina, Linwood Davidson County]",[],"[year 1865, 12 months, 4 months]",[Woodland Pasture],[Plantation],"[296.40, 8.75, 305.15, 111.00, 10.50, 121.50, ..."
3,Mem A Purchases November per Cooks books 55.20...,"[Cooks, Cook, Westerling]",[],"[Amt, Amt of Sparks]","[November, December, Dec 31st, January, Februa...",[],[],"[55.20, 53.20, 108.40, 73.75, 25, 98.75, 9.65,..."
4,Apparent indebtedness paid 94.19 Deduct Balanc...,"[Westerling, W.]",[],[Grocers],"[Feb 31.17, Nov December, two months]",[],[],"[94.19, 63.02, 25.15, 3, 25, 75.00, 49.85, 112..."


# Post processing for dates

In [23]:
recordF_onto["Date-ontonotes"]

0       [Quarter ending March 31st 1866, Dec 18, Jany ...
1                                                      []
2                        [year 1865, 12 months, 4 months]
3       [November, December, Dec 31st, January, Februa...
4                   [Feb 31.17, Nov December, two months]
                              ...                        
3594                                                   []
3595                                                   []
3596                                                   []
3597                                               [1895]
3598                                 [Sept 2565, Jan 166]
Name: Date-ontonotes, Length: 3599, dtype: object

In [24]:
G=[]
for i in recordF_onto["Date-ontonotes"]:
    M=[]
    for j in i:
        date=re.search(r'^\d\.*.*\d\d$' ,j)
        date2=re.search(r'^[A-Z]\.*.*\d\.*.*\d\d$' ,j)
        M.append(date)
        M.append(date2)
    G.append(M)
    

GF=[]
for i in G:
    Final_date=[]
    for j in i:
        if j!=None:
            Final_date.append(j.group())
    GF.append(Final_date)
    
            
      

In [25]:
len(GF)

3599

In [26]:
recordF_onto["postDate-ontonotes"]=GF

In [27]:
G=[]
for i in recordF_onto["postDate-ontonotes"]:
    M=[]
    for j in i:
        Final_date1=re.sub(r'([0-9]{1,2})\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        Final_date2=re.sub(r'([A-Z]\.*.*)\s([0-9]{1,2})\.*.*\s([0-9]{2,4})',r'\2-\1-\3' ,j)
        final_date3=re.sub(r'([A-Z][a-z]\S*)\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        final_date4=re.sub(r'(^[A-Za-z]\S*.*.)day\.*.*\s([A-Z][a-z]\S*)\.*.*\s([0-9]{2,4})',r'\1-\2-\3' ,j)
        if Final_date1 != j:   
            M.append(Final_date1)
        if Final_date2 != j :
            M.append(Final_date2)
        if final_date3 != j :
            M.append(final_date3)
        if final_date4 != j :
            M.append(final_date4)
    G.append(M)
    

recordF_onto["postDate-ontonotes"]=G

# standardizing dates

In [28]:
NG=[]
for i in G:
    ng=[]
    for j in i:
        
        date=re.findall(r'([0-9]{1,2})-([A-Z][a-z]\S*)-([0-9]{2,4})',j)
        if date !=[]:
            dates=[]
            for k in date:
                for l in k:
                    dates.append(l)
                
                
                if dates[1] in ["August","Aug","augt","AUGUST","Augst","Augt","AUG","Augusta"]:
                    dates[1]= "August"
                elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER",'Sepr','Septh']:
                    dates[1]= "Septembre"
                elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                    dates[1]= "October"
                elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER", "The"]:
                    dates[1]= "November"
                elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                    dates[1]= "December"
                elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                    dates[1]= "January"
                elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                    dates[1]= "February"
                elif dates[1] in ["March","Mar"]:
                    dates[1]= "March"
                elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                    dates[1]= "April"
                elif dates[1] in ["May","MAY"]:
                    dates[1]= "May"
                elif dates[1] in ["JUN","Jun","June"]:
                    dates[1]= "June"
                elif dates[1] in ["JULY","July","JUL"]:
                    dates[1]= "July"
                j = dates[0]+"-"+dates[1]+"-"+dates[2]
                ng.append(j)
        else :
                date = re.findall(r'(^[A-Za-z]\S*.*.)-([A-Z][a-z]\S*)-([0-9]{2,4})',j ) 
                dates=[]
                for k in date:
                    for l in k:
                        dates.append(l)
                
                
                    if dates[1] in ["August","Aug","augt","AUGUST","Augst",'Augt',"AUG","Augusta"]:
                        dates[1]= "August"
                    elif dates[1] in ["September","Sept","SEP","Sep","Septr","SEPTEMBER","Sepr","Septh", "This", "This "]:
                        dates[1]= "Septembre"
                    elif dates[1] in ["October","Oct","OCT","OCTOBER"]:
                        dates[1]= "October"
                    elif dates[1] in ["November","NOV","Nov","Novmbr","Nvbr","Novb","NOVEMBER"]:
                        dates[1]= "November"
                    elif dates[1] in ["December","Dec","Decbr","DEC","DECEMBER","Decber"]:
                        dates[1]= "December"
                    elif dates[1] in ["January","Janry","Jan","Jany","JAN","Janur","JANUARY"]:
                        dates[1]= "January"
                    elif dates[1] in ["February","Feby","Febry","FEB","FEBRUARY"]:
                        dates[1]= "February"
                    elif dates[1] in ["March","Mar"]:
                        dates[1]= "March"
                    elif dates[1] in ["April","Apl","Aprl","APRIL","Apr"]:
                        dates[1]= "April"
                    elif dates[1] in ["May","MAY"]:
                        dates[1]= "May"
                    elif dates[1] in ["JUN","Jun","June"]:
                        dates[1]= "June"
                    elif dates[1] in ["JULY","July","JUL"]:
                        dates[1]= "July"
                    if dates[0]== "fifteenth" :
                        dates[0]="15"
                    elif (dates[0]=="sixteenth") |(dates[0]=="This 16 "):
                        dates[0]="16"
                    elif dates[0]=="Twenty":
                        dates[0]="20" 
                    elif dates[0]=="Twentyseventh":
                        dates[0]="27" 
                    elif (dates[0]=="Twenty Eighth") |(dates[0]=="TwentyEight") |(dates[0]=="TwentyEight ") |(dates[0]=="Twentyeight"):
                        dates[0]="28"
                    elif (dates[0]=="Ninth") |(dates[0]=="Ninth ")|(dates[0]=="Augusta 9th ") |(dates[0]=="ninth") |(dates[0]=="ninth ")|(dates[0]=="Augusta 9th"):
                        dates[0]="9"
                    
                    elif dates[0]=='Seventh':
                        dates[0]="7" 
                    elif (dates[0]=='The') |(dates[0]=="The 9th") :
                        dates[0]="9"
                    elif dates[0]=="the 9" :
                        dates[0]="9"
                    elif (dates[0]=="Eighth ") |(dates[0]=="Eighth") |(dates[0]=="eighth") |(dates[0]=="eighth "):
                        dates[0]="8"
                    elif dates[0]=="Seventh ":
                        dates[0]="7"
                    elif (dates[0]=="Twenty ninth ")| (dates[0]=="Twenty ninth") | (dates[0]=="twenty ninth ")|(dates[0]=="Twenty Ninth ")|(dates[0]=="Twenty Ninth") :
                        dates[0]="29"
                    elif (dates[0]=="Thirty") |(dates[0]=="Thirty ") | (dates[0]=="thirtieth "):
                        dates[0]="30"
                    elif (dates[0]=="Thirty first ") |(dates[0]=="Thirty first"):
                        dates[0]="31"
                  
                    
                    elif dates[0]=="first ":
                        dates[0]="1"
                    elif (dates[0]=="Second ") |(dates[0]=="Second") |(dates[0]=="second")|(dates[0]=="second "):
                        dates[0]="2"
                    elif dates[0]=="fourth ":
                        dates[0]="4"
                    elif dates[0]=="tenth ":
                        dates[0]="10"
                    elif (dates[0]=="Fifth ")|(dates[0]=="Fifth") |(dates[0]=="fifth ")|(dates[0]=="fifth"):
                        dates[0]="5"
                    elif (dates[0]=="This") | (dates[0]=="This 7th ")|(dates[0]=="seventh ") |(dates[0]=="seventh") |(dates[0]=="Seventh "):
                        dates[0]="7"
                    elif (dates[0]=="This 31st") |(dates[0]=="Thirty-first "):
                        dates[0]="31"
                    elif (dates[0]=="Thirtieth") |(dates[0]=="Thirtieth ") |(dates[0]=="Thirty"):
                        dates[0]="30"
                    elif dates[0]=="twenty fifth ":
                        dates[0]="25"
                    elif dates[0]=="Twenty seventh ":
                        dates[0]="27"
                    elif (dates[0]=="Twenty sixth ") |(dates[0]=="Twentysixth ")|(dates[0]=="Twentysixth") |(dates[0]=="twentysixth "):
                        dates[0]="26"
                    elif dates[0]=="third ":
                        dates[0]="3" 
                    elif dates[0]=="third":
                        dates[0]="3"
                    elif dates[0]=="Third ":
                        dates[0]="3"
                    elif dates[0]=="Third":
                        dates[0]="3"
                    
                    elif dates[0]=="Tenth":
                        dates[0]="10" 
                    elif dates[0]=="Augusta":
                        dates[0]="9" 
                    elif (dates[0]=="this_sixth ")|(dates[0]=="sixth ")| (dates[0]=="Sixth") | (dates[0]=="Sixth ") |(dates[0]=="this sixth "):
                        dates[0]="6" 
                        
                    elif dates[0]=="Tenth ":
                        dates[0]="10"
                    elif (dates[0]=="twentieth")|(dates[0]=="This 20th "):
                        dates[0]="20"
                    elif (dates[0]=="twenty fourth ") |(dates[0]=="twenty fourth") |(dates[0]=="Twenty fourth ") |(dates[0]=="Twenty fourth"):
                        dates[0]="24"
                    elif (dates[0]=="fifteenth ") |(dates[0]=="Fifteenth ") |(dates[0]=="Fifteenth"):
                        dates[0]="15"
                    elif dates[0]=="twentieth ":
                        dates[0]="20"
                    elif dates[0]=="Eighteenth ":
                        dates[0]="18"
                    elif dates[0]=="Eighteenth":
                        dates[0]="18"
                    elif (dates[0]=="Nineteenth ") |(dates[0]=="Nineteenth"):
                        dates[0]="19"
                    elif dates[0]=="First ":
                        dates[0]="1"
                    elif dates[0]=="First":
                        dates[0]="1"
                    elif (dates[0]=="Sixteenth ")|(dates[0]=="this 16") |(dates[0]=="Sixteenth"):
                        dates[0]="16"
                   
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif dates[0]=="twenty third":
                        dates[0]="23"
                    elif dates[0]=="Twenty third ":
                        dates[0]="23"
                    elif dates[0]=="Twenty Second ":
                        dates[0]="22"
                    elif dates[0]=="Twenty Second":
                        dates[0]="22"
                    elif dates[0]=="Twenty Eighth ":
                        dates[0]="28"
                    elif dates[0]=="Twentysenventh ":
                        dates[0]="27"
                    elif dates[0]=="twenty third ":
                        dates[0]="23"
                    elif (dates[0]=="twenty third") |(dates[0]=="Twentythird")|(dates[0]=="Twentythird "):
                        dates[0]="23"
                    elif (dates[0]=="fourteenth ") | (dates[0]=="fourteenth") | (dates[0]=="Fourteenth") | (dates[0]=="Fourteenth "):
                        dates[0]="14"
                    elif (dates[0]=="Eleventh ") | (dates[0]=="Eleventh") |(dates[0]=="eleventh ") :
                        dates[0]="11"
                    elif (dates[0]=="Thirteenth") | (dates[0]=="Thirteenth ") :
                        dates[0]="13"
                    elif (dates[0]=="seventeenth ") |(dates[0]=="seventeenth") |(dates[0]=="Seventeenth ") :
                        dates[0]="17"
                     
                     
                     
                    
                        
                    j = dates[0]+"-"+dates[1]+"-"+dates[2]
                    ng.append(j)
            
            
    NG.append(ng)   

In [29]:
recordF_onto["standard_date-ontonotes"]=NG

In [30]:
recordF_onto.to_csv("recordF_onto.csv",index=False)

# Ner english large 

In [31]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

recordF_large=pd.DataFrame({"recordF_text":"", "Name-nerenglarge":[], "LOC-nerenglarge":"","ORG-nerenglarge":""})
for i in range(3599):
        sentencess =Sentence(recordF_text[i])
        recordF_large.loc[i,"recordF_text"]=recordF_text[i]
        tagger2.predict(sentencess)
        dc={}
        P=[]
        L=[]
        G=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG") :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

recordF_large["Name-nerenglarge"]=Per
recordF_large["LOC-nerenglarge"]=LOC
recordF_large["ORG-nerenglarge"]=ORG    
recordF_large["other_namelarge"] =Other_name 

In [32]:
recordF_large.head()

,recordF_text,Name-nerenglarge,LOC-nerenglarge,ORG-nerenglarge,other_namelarge
0,Quarter ending March 31st 1866 Justin Hodge Ca...,"[Justin Hodge, Henry Swim, Wm B Meins, JC Barn...","[Dist West, N.Ca, Greensboro N.C, Nottsburg, S...","[A.Q.M . Financial Agent Bureau, Fines, AQM, U...",[]
1,Report of Monies recd Dist W NC previously rep...,"[Cilley, Wm McFarlane, Dag Palmer, Clinton A C...",[NC],[AAG],[]
2,N. Carolina Memorandum of Expenses Linwood Dav...,"[G . W. Myers, R . Lanier, C . Davis, W.R . Holt]","[N. Carolina, Linwood Davidson County]",[],[White]
3,Mem A Purchases November per Cooks books 55.20...,[Dr Westerling],[],"[Cooks, Cook, Amt of Cooks, Amt of Sparks]",[]
4,Apparent indebtedness paid 94.19 Deduct Balanc...,"[W, Westerling, W.]",[],[],[]


In [33]:
recordF_large.to_csv("recordF_large.csv",index=False)

# Ner english

In [34]:
lst=[]
label=[]
Per=[]
ORG=[]
LOC=[]
Other_name=[]
from flair.data import Sentence

recordF_ner=pd.DataFrame({"recordF_text":"", "PERSON-nerenglish":"","LOC-nerenglish":"","ORG-nerenglish":[]})
for i in range(20):
        sentencess =Sentence(recordF_text[i])
        recordF_ner.loc[i,"recordF_text"]=recordF_text[i]
        tagger1.predict(sentencess)
        dc={}
        P=[]
        L=[]
        O=[]
        Other=[]
        for entity in sentencess.get_spans('ner'):
                     
                     if (entity.get_label("ner").score > 0.90) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                        dc[entity.text]= entity.get_label("ner").value 
        
        
        for j in dc :
                if dc[j]=="PER":
                      P.append(j)
                elif (dc[j]=="ORG")  :
                      O.append(j)
                elif  (dc[j]=="MISC") :
                      Other.append(j)
                elif dc[j]=="LOC":
                      L.append(j)
               
        Per.append(P)
        ORG.append(O)
        LOC.append(L)
        Other_name.append(Other)
            

recordF_ner["Name-nerenglish"]=Per
recordF_ner["LOC-nerenglish"]=LOC
recordF_ner["ORG-nerenglish"]=ORG 
recordF_ner["Other_name-nerenglish"]=Other_name 

In [35]:
recordF_ner.head()

,recordF_text,PERSON-nerenglish,LOC-nerenglish,ORG-nerenglish,Name-nerenglish,Other_name-nerenglish
0,Quarter ending March 31st 1866 Justin Hodge Ca...,NaN,[],[],"[Jacob Bodecker, Justin Hodge]",[]
1,Report of Monies recd Dist W NC previously rep...,NaN,[],[],[],[]
2,N. Carolina Memorandum of Expenses Linwood Dav...,NaN,[Woodland Pasture],[],[],[]
3,Mem A Purchases November per Cooks books 55.20...,NaN,[],[],[],[]
4,Apparent indebtedness paid 94.19 Deduct Balanc...,NaN,[],[],[],[]


In [36]:
recordF_ner.to_csv("recordF_ner.csv",index=False)

In [37]:
recordF_onto["Name-nerenglarge"]=recordF_large["Name-nerenglarge"]
recordF_onto["LOC-nerenglarge"]=recordF_large["LOC-nerenglarge"]
recordF_onto["ORG-nerenglarge"]=recordF_large["ORG-nerenglarge"]
recordF_onto["other_namelarge"]=recordF_large["other_namelarge"]
recordF_onto["PERSON-nerenglish"]=recordF_ner["Name-nerenglish"]
recordF_onto["LOC-nerenglish"]=recordF_ner["LOC-nerenglish"]
recordF_onto["ORG-nerenglish"]=recordF_ner["ORG-nerenglish"]
recordF_onto["Other_name-nerenglish"]=recordF_ner["Other_name-nerenglish"]

In [38]:
to_compare_result_recordF=recordF_onto

In [39]:
to_compare_result_recordF.to_csv("to_compare_result_recordF.csv", index=False)